In [1]:
import pandas as pd
import numpy as np
import os
import nltools as nlt
import nilearn as nil
import nibabel as nib
import warnings
import glob
import random
import pickle
import dev_wtp_io_utils
import gc #garbage collection
from nilearn import plotting


/home/bsmith16/.conda/envs/neuralsignature/lib/python3.8/site-packages/nilearn/datasets/__init__.py:87: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [2]:
pd.set_option('display.max_rows', 99)

### Load brain data

In [3]:
test_train_set = pd.read_csv("../data/train_test_markers_20210601T183243.csv")

In [4]:
with open('../data/Brain_Data_2sns_60subs.pkl', 'rb') as pkl_file:
    Brain_Data_allsubs = pickle.load(pkl_file)
    
dev_wtp_io_utils.check_BD_against_test_train_set(Brain_Data_allsubs,test_train_set)

checked for intersection and no intersection between the brain data and the subjects was found.
there were 60 subjects overlapping between the subjects marked for train data and the training dump file itself.


In [5]:
with open('../data/Brain_Data_2sns_60subs_grouped.pkl', 'rb') as pkl_file:
    Brain_Data_allsubs_grouped = pickle.load(pkl_file)
    
dev_wtp_io_utils.check_BD_against_test_train_set(Brain_Data_allsubs_grouped,test_train_set)

checked for intersection and no intersection between the brain data and the subjects was found.
there were 60 subjects overlapping between the subjects marked for train data and the training dump file itself.


### Preprocess

In [6]:
Brain_Data_allsubs.Y = Brain_Data_allsubs.X.response.copy()
print(Brain_Data_allsubs.Y.value_counts())
Brain_Data_allsubs.Y[Brain_Data_allsubs.Y=='NULL']=None
print(Brain_Data_allsubs.Y.value_counts())
print(Brain_Data_allsubs.Y.isnull().value_counts())
Brain_Data_allsubs_nn = Brain_Data_allsubs[Brain_Data_allsubs.Y.isnull()==False]
print(len(Brain_Data_allsubs_nn))
print(len(Brain_Data_allsubs))

5.0    1164
6.0    1018
7.0     904
8.0     604
Name: response, dtype: int64
5.0    1164
6.0    1018
7.0     904
8.0     604
Name: response, dtype: int64
False    3690
True      150
Name: response, dtype: int64
3690
3840


In [7]:
all_subs_nn_nifti = Brain_Data_allsubs_nn.to_nifti()
all_subs_nn_nifti_Y = Brain_Data_allsubs_nn.Y
all_subs_nn_nifti_groups = Brain_Data_allsubs_nn.X.subject
all_subs_nn_nifti_groups

0       DEV001
1       DEV001
2       DEV001
3       DEV001
4       DEV001
         ...  
3685    DEV089
3686    DEV089
3687    DEV089
3688    DEV089
3689    DEV089
Name: subject, Length: 3690, dtype: object

In [8]:
Brain_Data_allsubs_grouped.Y = Brain_Data_allsubs_grouped.X.response.copy()
print(Brain_Data_allsubs_grouped.Y.value_counts())
all_subs_grouped_nifti = Brain_Data_allsubs_grouped.to_nifti()
all_subs_grouped_nifti_Y = Brain_Data_allsubs_grouped.Y
all_subs_grouped_nifti_groups = Brain_Data_allsubs_grouped.X.subject
all_subs_grouped_nifti_groups

6.0    236
7.0    235
5.0    227
8.0    202
Name: response, dtype: int64


0      DEV001
1      DEV001
2      DEV001
3      DEV001
4      DEV001
        ...  
895    DEV089
896    DEV089
897    DEV089
898    DEV089
899    DEV089
Name: subject, Length: 900, dtype: object

### Predict

Regressing in nilearn:
 - https://nilearn.github.io/decoding/estimator_choice.html
 - http://www.ncbi.nlm.nih.gov/pubmed/20691790







OK, so that's how you do it. It's pretty straightforward.

So...we won't look at nested cross-validation juuust yet, because the next step is to work out how to train on one set and predict on another. that will definitely require a custom pipeline. Let's get started...

In [9]:
del Brain_Data_allsubs
del Brain_Data_allsubs_grouped
gc.collect()

15

In [10]:
from nilearn.decoding import DecoderRegressor
dRegressor = DecoderRegressor(estimator = 'ridge_regressor', standardize= True,scoring="r2")

In [12]:
import sys
for name, size in sorted(((name, sys.getsizeof(value)) for name, value in locals().items()),
                         key= lambda x: -x[1])[:10]:
    print(name + ': ' + str(size))

___: 232614
all_subs_nn_nifti_groups: 232614
_7: 232614
__: 56844
all_subs_grouped_nifti_groups: 56844
_8: 56844
all_subs_nn_nifti_Y: 29664
test_train_set: 9549
all_subs_grouped_nifti_Y: 7344
DecoderRegressor: 1472


As a control, we'll try this again, this time just training and testing on individual values:

In [13]:
from pympler.asizeof import asizeof

def asizeof_fmt(obj, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    num = asizeof(obj)
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

# for name, size in sorted(((name, asizeof(value)) for name, value in locals().items()),
#                          key= lambda x: -x[1])[:10]:


In [14]:
asizeof_fmt(Brain_Data_allsubs_nn)

'3.3 GiB'

In [15]:
asizeof_fmt(all_subs_nn_nifti)

'12.4 GiB'

get a small sample of subjects to extract

In [16]:
sample_subject_items = np.unique(all_subs_nn_nifti_groups)[0:15]

In [17]:
sample_subject_vector = [i for i, x in enumerate(all_subs_nn_nifti_groups) if x in sample_subject_items]
sample_grouped_subject_vector = [i for i, x in enumerate(all_subs_grouped_nifti_groups) if x in sample_subject_items]

In [18]:
len(sample_subject_vector)

897

and now extract them

In [19]:
first_subs_nifti = nib.funcs.concat_images([all_subs_nn_nifti.slicer[...,s] for s in sample_subject_vector])
first_subs_nifti_Y = all_subs_nn_nifti_Y[sample_subject_vector]
first_subs_nifti = nil.image.clean_img(first_subs_nifti,detrend=False,standardize=True)
first_subs_nifti_groups = all_subs_nn_nifti_groups[sample_subject_vector]

del all_subs_nn_nifti
gc.collect()

89

In [21]:
first_subs_nifti.shape

(91, 109, 91, 897)

In [22]:
first_subs_grouped_nifti = nib.funcs.concat_images([all_subs_grouped_nifti.slicer[...,s] for s in sample_grouped_subject_vector])
first_subs_grouped_nifti_Y = all_subs_grouped_nifti_Y[sample_grouped_subject_vector]
first_subs_grouped_nifti = nil.image.clean_img(first_subs_grouped_nifti,detrend=False,standardize=True)
first_subs_grouped_nifti_groups = all_subs_grouped_nifti_groups[sample_grouped_subject_vector]

del all_subs_grouped_nifti
gc.collect()

In [24]:
first_subs_grouped_nifti.shape

(91, 109, 91, 223)

In [29]:
from sklearn.model_selection import KFold,GroupKFold

In [41]:

def cv_train_test_different_sets(
    individual_X,individual_groups, individual_y, 
    averaged_X = None,averaged_y = None, averaged_groups = None,
    cv = None,regressorGenerator=None):
    """
    averaged_X: values grouped
    averaged_groups: group allocations for the averaged dataset
    individual_X: values grouped into averages for testing
    cv: a Grouped cross-validator
    group_list: name of the groups
    """
    if cv is None:
        cv=KFold(n_splits=5)

    groups_array = np.array(list(set(individual_groups)))
    assert(set(averaged_groups)==set(individual_groups))

    #the CV that the inner Regressor uses
    cv_inner = GroupKFold(2)

    #we actually use KFold on the group names themselves, then filter across that
    #that's equivalent to doing a GroupedKFold on the data.
    test_scores = []
    for train_i,test_i in cv.split(groups_array):
        train_group_items, test_group_items = groups_array[train_i], groups_array[test_i]
        print('In order to test on a training group of ' +
              str(len(train_group_items)) + ' items, holding out the following subjects:' +
              str(test_group_items),end='. ')
        
        
        #select training data from the averages
        print('selecting training data',end='. ')
        train_selector = [i for i, x in enumerate(averaged_groups) if x in train_group_items]
        train_y = averaged_y[train_selector]
        train_X = nib.funcs.concat_images([averaged_X.slicer[...,s] for s in train_selector])
        train_groups = averaged_groups[train_selector]
        print(train_X.shape,end='. ')
        print(asizeof_fmt(train_X),end='. ')

        #select testing data from the individual values
        print('selecting test data',end='. ')
        test_selector = [i for i, x in enumerate(individual_groups) if x in test_group_items]
        test_y = individual_y[test_selector]
        test_X = nib.funcs.concat_images([individual_X.slicer[...,s] for s in test_selector])
        test_groups = individual_groups[test_selector]
        print(asizeof_fmt(test_X),end='. ')
        print(test_X.shape,end='. ')


        print("regressing",end='. ')
        regressor = DecoderRegressor(standardize= True,cv=cv_inner, scoring="r2")
        print(asizeof_fmt(train_X),end='. ')
        regressor.fit(y=train_y,X=train_X,groups=train_groups)

        print("predicting",end='. ')
        #now predict on our test split
        test_score = regressor.score(test_X,test_y)
        test_scores = test_scores+[test_score]
        print('test score was:',end='. ')
        print(test_score)

        del test_X
        del train_X
        gc.collect() #clean up. this is big data we're working with
        #https://stackoverflow.com/questions/1316767/how-can-i-explicitly-free-memory-in-python

    return(test_scores)





In [42]:
def cv_train_test_same_sets(
    individual_X,individual_groups, individual_y, 
    #averaged_X = None,averaged_y = None, averaged_groups = None,
    cv = None):
    """
    averaged_X: values grouped
    averaged_groups: group allocations for the averaged dataset
    individual_X: values grouped into averages for testing
    cv: a Grouped cross-validator
    """
    #individual_X = first_subs_nifti#all_subs_nn_nifti
    #individual_y = first_subs_nifti_Y #all_subs_nn_nifti_Y
    #individual_groups = all_subs_nn_nifti_groups[0:500]
    if cv is None:
        cv=KFold(n_splits=5)



    groups_array = np.array(list(set(individual_groups)))

    #the CV that the inner Regressor uses
    cv_inner = GroupKFold(2)

    #we actually use KFold on the group names themselves, then filter across that
    #that's equivalent to doing a GroupedKFold on the data.
    test_scores = []
    for train_i,test_i in cv.split(groups_array):
        train_group_items, test_group_items = groups_array[train_i], groups_array[test_i]
        print('In order to test on a training group of ' +
              str(len(train_group_items)) + ' items, holding out the following subjects:' +
              str(test_group_items),end='. ')

        #select training data from the averages
        print('selecting training data',end='. ')
        train_selector = [i for i, x in enumerate(individual_groups) if x in train_group_items]
        train_y = individual_y[train_selector]
        train_X = nib.funcs.concat_images([individual_X.slicer[...,s] for s in train_selector])
        train_groups = individual_groups[train_selector]
        print(asizeof_fmt(train_X),end='. ')
        print(train_X.shape,end='. ')

        #select testing data from the individual values
        print('selecting test data',end='. ')
        test_selector = [i for i, x in enumerate(individual_groups) if x in test_group_items]
        test_y = individual_y[test_selector]
        test_X = nib.funcs.concat_images([individual_X.slicer[...,s] for s in test_selector])

        test_groups = individual_groups[test_selector]
        print(test_X.shape,end='. ')

        print("regressing",end='. ')
        #regressor = DecoderRegressor(standardize= True,cv=cv_inner, scoring="r2")
        regressor = DecoderRegressor(standardize= True,cv=cv_inner, scoring="r2")
        print(asizeof_fmt(train_X),end='. ')
        regressor.fit(y=train_y,X=train_X,groups=train_groups)

        print("predicting",end='. ')
        #now predict on our test split
        test_score = regressor.score(test_X,test_y)
        test_scores = test_scores+[test_score]
        print('test score was:',end='. ')
        print(test_score)

        del test_X
        del train_X
        gc.collect() #clean up. this is big data we're working with
    return(test_scores)
        #https://stackoverflow.com/questions/1316767/how-can-i-explicitly-free-memory-in-python

In [43]:
test_scores_same = cv_train_test_same_sets(
    individual_X=first_subs_nifti,
    individual_y=first_subs_nifti_Y,
    individual_groups=first_subs_nifti_groups
)

print(test_scores_same)

In order to test on a training group of 12 items, holding out the following subjects:['DEV010' 'DEV019' 'DEV021']. selecting training data. 4.9 GiB. (91, 109, 91, 733). selecting test data. (91, 109, 91, 164). regressing. 4.9 GiB. predicting. test score was:. 0.21853039978740596
In order to test on a training group of 12 items, holding out the following subjects:['DEV006' 'DEV012' 'DEV005']. selecting training data. 4.8 GiB. (91, 109, 91, 711). selecting test data. (91, 109, 91, 186). regressing. 4.8 GiB. predicting. test score was:. 0.21890934756666858
In order to test on a training group of 12 items, holding out the following subjects:['DEV013' 'DEV017' 'DEV001']. selecting training data. 4.8 GiB. (91, 109, 91, 720). selecting test data. (91, 109, 91, 177). regressing. 4.8 GiB. predicting. test score was:. 0.09647748341542672
In order to test on a training group of 12 items, holding out the following subjects:['DEV022' 'DEV009' 'DEV015']. selecting training data. 4.8 GiB. (91, 109, 9

NameError: name 'test_scores' is not defined

In [44]:
print(test_scores_same)

[0.21853039978740596, 0.21890934756666858, 0.09647748341542672, 0.16768299712319068, 0.26106811642073835]


In [63]:
np.mean(test_scores_same)

0.12399989268473066

In [46]:
test_scores = cv_train_test_different_sets(
    averaged_X=first_subs_grouped_nifti,
    averaged_y=first_subs_grouped_nifti_Y,
    averaged_groups=first_subs_grouped_nifti_groups,
    individual_X=first_subs_nifti,
    individual_y=first_subs_nifti_Y,
    individual_groups=first_subs_nifti_groups)

In order to test on a training group of 12 items, holding out the following subjects:['DEV010' 'DEV019' 'DEV021']. selecting training data. (91, 109, 91, 179). 1.2 GiB. selecting test data. 1.1 GiB. (91, 109, 91, 164). regressing. 1.2 GiB. predicting. test score was:. 0.1873604351084126
In order to test on a training group of 12 items, holding out the following subjects:['DEV006' 'DEV012' 'DEV005']. selecting training data. (91, 109, 91, 178). 1.2 GiB. selecting test data. 1.3 GiB. (91, 109, 91, 186). regressing. 1.2 GiB. predicting. test score was:. 0.09609407198510678
In order to test on a training group of 12 items, holding out the following subjects:['DEV013' 'DEV017' 'DEV001']. selecting training data. (91, 109, 91, 181). 1.2 GiB. selecting test data. 1.2 GiB. (91, 109, 91, 177). regressing. 1.2 GiB. predicting. test score was:. -0.07183434134739297
In order to test on a training group of 12 items, holding out the following subjects:['DEV022' 'DEV009' 'DEV015']. selecting training

In [47]:
test_scores

[0.1873604351084126,
 0.09609407198510678,
 -0.07183434134739297,
 0.07951115382986251,
 0.12834946390442026]

In [48]:
np.mean(test_scores)

0.08389615669608183

#### Nested cross-validation

See for instance: http://nilearn.github.io/auto_examples/02_decoding/plot_haxby_grid_search.html
        
See also https://machinelearningmastery.com/nested-cross-validation-for-machine-learning-with-python/

and this thread: https://neurostars.org/t/nested-cross-validation/19459

The Haxby code:

(1) Loops through each outer loop
(2) And for each outer loop, loops again through each parameter
(3) And then estimates three times on the _training_ data for that
(4) Then gets a prediction score for that hyperparameter configuration

This is fundamentally the same as the nested CV in Machine Learning Mastery, with one exception. The Haxby code calculates a prediction on the test set once per hyperparameter. The MLM code calculates a prediction on the test set once per outer split. There is more stuff going on in the GridSearchCV but you don't see that.

The Decoder object actually as a param_grid command so this could be useful for doing the parameter search properly, in the inner loop.


Issues here:

 - Need to get the cross-validation right. We have got it working but there is an inner validation that selects a decoder for each group. Does that make sense? I think so, but just need to consider it a bit carefully.
 - What are the individual methods that inner validation is running? Should keep track of that
 - We really need to compare this to a baseline. So we need to run an individual-level analysis
